In [46]:
import numpy as np
import matplotlib
import cv2
import torch
from torchvision import models
from PIL import Image, ImageStat
import matplotlib.pyplot as plt
import imutils
import math
from scipy.signal import convolve2d
import torchvision.transforms as Tensor
import glob
import csv

filename = 'C:/Users/Akhil Jalla/Desktop/Final Project/Final_part2.csv'

deeplabv3 = models.segmentation.deeplabv3_resnet101(pretrained=1).eval()
preprocess = Tensor.Compose([Tensor.Resize(256),
#                  Tensor.CenterCrop(512),
                 Tensor.ToTensor(), 
                 Tensor.Normalize(mean = [0.485, 0.456, 0.406], 
                             std = [0.229, 0.224, 0.225])])

In [47]:
images = []
for f in glob.glob("C:/Users/Akhil Jalla/Desktop/Final Project/bad images/*.jpg"):
    image = cv2.imread(f)
    images.append(image)
print(len(images))

800


In [48]:
def decode_segmap(image, nc=21):
    label_colors = np.array([(255, 255, 255),  # 0=background
               # 1=aeroplane, 2=bicycle, 3=bird, 4=boat, 5=bottle
               (255, 255, 255), (255, 255, 255), (255, 255, 255), (255, 255, 255), (255, 255, 255),
               # 6=bus, 7=car, 8=cat, 9=chair, 10=cow
               (255, 255, 255), (255, 255, 255), (255, 255, 255), (255, 255, 255), (255, 255, 255),
               # 11=dining table, 12=dog, 13=horse, 14=motorbike, 15=person
               (255, 255, 255), (255, 255, 255), (255, 255, 255), (255, 255, 255), (0, 0, 0),
               # 16=potted plant, 17=sheep, 18=sofa, 19=train, 20=tv/monitor
               (255, 255, 255), (255, 255, 255), (255, 255, 255),(255, 255, 255), (255, 255, 255)])

    r = np.zeros_like(image).astype(np.uint8)
    g = np.zeros_like(image).astype(np.uint8)
    b = np.zeros_like(image).astype(np.uint8)
  
    for l in range(0, nc):
        idx = image == l
        r[idx] = label_colors[l, 0]
        g[idx] = label_colors[l, 1]
        b[idx] = label_colors[l, 2]
    
    rgb = np.stack([r, g, b], axis=2)
    return rgb

def deeplab(img):
    img_pil = Image.fromarray(img)
    inp = preprocess(img_pil).unsqueeze(0)
    output = deeplabv3(inp)['out']
#    print(out.shape)
    tensor_result = torch.argmax(output.squeeze(), dim=0).detach().cpu().numpy()
    rgb1 = decode_segmap(tensor_result)
    return rgb1



In [49]:
def region_of_interest(img):
    number_of_black_pixels = np.count_nonzero(np.all(img==[0,0,0],axis=2))
    if number_of_black_pixels > 500:
        ROI = 1
        print('Region of interest is present i.e ROI = 1')
    else:
        ROI = 0
        print('Region of interest is not present')
    return ROI

def blurriness(img,img1):
    if ROI==1:
        segmented_person_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        background_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
        blur_score_foreground = cv2.Laplacian(segmented_person_gray, cv2.CV_64F).var()
        blur_score_background = cv2.Laplacian(background_gray, cv2.CV_64F).var()
        print('Blur score of the foreground is ',blur_score_foreground)
        print('Blur score of the background is ',blur_score_background)
    else:
        blur_score_foreground = 0.0
        blur_score_background = 0.0
        print('Doesnt have an ROI')
    return blur_score_foreground, blur_score_background

def brightness(img):
    if ROI==1:
        img_pil = Image.fromarray(img)
        stat = ImageStat.Stat(img_pil)
        r,g,b = stat.mean
        brightness_score = math.sqrt(0.241*(r**2) + 0.691*(g**2) + 0.068*(b**2))
        print('Brightness score of the image is ', brightness_score)
    else:
        brightness_score = 0.0
        print('Doesnt have an ROI')
    return brightness_score

def noise(img):
    if ROI==1:
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        h,w = img_gray.shape
        M = [[1, -2, 1],[-2, 4, -2],[1, -2, 1]]
        sigma_noise = np.sum(np.sum(np.absolute(convolve2d(img_gray, M))))
        sigma_noise = sigma_noise * math.sqrt(0.5 * math.pi) / (6 * (w-2) * (h-2))
        print('the noise score of the image is ', sigma_noise)
    else:
        sigma_noise = 0.0
        print('Doesnt have an ROI')
    return sigma_noise

def colorfulness(img):
    if ROI==1:
        (b,g,r) = cv2.split(img.astype('float'))
        rg = np.absolute(r-g)
        yb = np.absolute((0.5)*(r+g)-b)
        rbMean = np.mean(rg)
        ybMean = np.mean(yb)
        rbStd = np.std(rg)
        ybStd = np.std(yb)

        stdRoot = np.sqrt((rbStd ** 2) + (ybStd ** 2))
        meanRoot = np.sqrt((rbMean ** 2) + (ybMean ** 2))
        colorfulness_score = stdRoot + (0.3 * meanRoot)
        print('Colourfullness score of the image is ', colorfulness_score)
    else:
        colorfulness_score = 0.0
        print('Doesnt have an ROI')
    return colorfulness_score

def intersection_points(x1,y1,x2,y2,x3,y3,x4,y4):
    x_intercept = ( (x1*y2-y1*x2)*(x3-x4)-(x1-x2)*(x3*y4-y3*x4) ) / ( (x1-x2)*(y3-y4)-(y1-y2)*(x3-x4) )
    y_intercept = ( (x1*y2-y1*x2)*(y3-y4)-(y1-y2)*(x3*y4-y3*x4) ) / ( (x1-x2)*(y3-y4)-(y1-y2)*(x3-x4) )
    return x_intercept, y_intercept 

def contrast(img):
    if ROI==1:
        img_pil = Image.fromarray(img)
        h,w = img_pil.size
        img_pil_array = np.array(img_pil)
        intensity = 0
        for i in range(img_pil_array.shape[0]):
            for j in range (img_pil_array.shape[1]):
                intensity += (img_pil_array[i,j,0] - brightness_score)**2
    #     print(intensity)
        total_pixel = h * w
        contrast_score = math.sqrt(intensity/total_pixel)
        print('the contrast score of the image is ',contrast_score)
    else:
        contrast_score = 0.0
        print('Doesnt have an ROI')
    return contrast_score

def ROT(img):
    if ROI ==1:
        new_image = np.copy(img)
        new_image[np.where((new_image == [255,255,255]).all(axis = 2))] = [125,125,125]
        new_image[np.where((new_image == [0,0,0]).all(axis = 2))] = [255,255,255]
        new_image[np.where((new_image == [125,125,125]).all(axis = 2))] = [0,0,0]
        gray_image = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)
        ret,thresh = cv2.threshold(gray_image,127,255,cv2.THRESH_BINARY)
        contours = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)[-2]
        countour_count = 0
        for c in contours:
            countour_count += 1

        M = cv2.moments(contours[0])
        # calculate x,y coordinate of centeroid
        if M["m00"]==0:
            cX = 0
            cY = 0    
        else:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])

        abc = cv2.circle(new_image, (cX, cY), 5, (125, 125, 125), -1)
        width, height, channel = abc.shape
        horizontal_x1 = int((1/3)*height)
        horizontal_x2 = int((2/3)*height)
        vertical_x1 = int((1/3)*width)
        vertical_x2 = int((2/3)*width)

        x1,y1 = intersection_points(horizontal_x1,0,horizontal_x1,width-1,0,vertical_x1,height-1,vertical_x1)
        x2,y2 = intersection_points(horizontal_x2,0,horizontal_x2,width-1,0,vertical_x1,height-1,vertical_x1)
        x3,y3 = intersection_points(horizontal_x1,0,horizontal_x1,width-1,0,vertical_x2,height-1,vertical_x2)
        x4,y4 = intersection_points(horizontal_x2,0,horizontal_x2,width-1,0,vertical_x2,height-1,vertical_x2)

        dist_ROT = min((np.sqrt((cX-x1)**2 + (cY-y1)**2)),(np.sqrt((cX-x2)**2 + (cY-y2)**2)),(np.sqrt((cX-x3)**2 + (cY-y3)**2)),(np.sqrt((cX-x4)**2 + (cY-y4)**2))) 
        contour_area = cv2.contourArea(contours[0])
        Area_ratio = contour_area/(width*height)

        ROT_score = ((contour_area*0.05)/(height*width))*(1-((dist_ROT)/(np.sqrt(height**2 + width**2))))
        print('Ratio of area of foreground to the area of image is ', Area_ratio)
        print('The rule of third score of the image is ', ROT_score)
    else:
        Area_ratio = 0.0
        ROT_score = 0.0
        print('Doesnt have an ROI')
    return ROT_score, Area_ratio

def potrait_or_landscape(img):
    width,height,channel = img.shape
    if width > height:
        potrait = 1
        landscape = 0
        print('the image is in potrait mode')
    elif height>width:
        potrait = 0
        landscape = 1
        print('the image is in landscape mode')
    elif height==width:
        potrait = 1
        landscape = 1
        print('the image is a square image')
    return potrait, landscape

def distance_from_center(img):
    if ROI==1:
        new_image = np.copy(img)
        new_image[np.where((new_image == [255,255,255]).all(axis = 2))] = [125,125,125]
        new_image[np.where((new_image == [0,0,0]).all(axis = 2))] = [255,255,255]
        new_image[np.where((new_image == [125,125,125]).all(axis = 2))] = [0,0,0]
        gray_image = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)
        ret,thresh = cv2.threshold(gray_image,127,255,cv2.THRESH_BINARY)
        contours = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)[-2]
        countour_count = 0
        for c in contours:
            countour_count += 1
        M = cv2.moments(contours[0])
        # calculate x,y coordinate of centeroid
        if M["m00"]==0:
            cX = 0
            cY = 0
            ratio = 100.0
        else:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            width, height, channel = new_image.shape
            X1 = int((1/2)*height)
            Y1 = 0
            X2 = int((1/2)*height)
            Y2 = int(width - 1)
            dist = distance(cX,cY,X1,Y1,X2,Y2)
            ratio = dist/(0.5*height)
            print('Ratio of distance of the centroid from the center to the half of width of the image is ',ratio)
    else:
        ratio = 0.0
        print('Doesnt have an ROI')
    return ratio

def distance(x0,y0,x1,y1,x2,y2):
    numerator = abs((y2 - y1) * x0 - (x2 - x1) * y0 + x2 * y1 - y2 * x1)
    denominator = ((y2 - y1)**2 + (x2 - x1) ** 2) ** 0.5
    result = numerator/denominator
    return result



In [5]:
# fileEmpty = os.path.isfile(filename) 

for i in range(len(images)):
    rgb = deeplab(images[i])
    cv_image = cv2.cvtColor(np.array(rgb), cv2.COLOR_RGB2BGR)
    h,w,n = cv_image.shape
    resized_image = cv2.resize(images[i],(w,h))
    
    mask = (cv_image == 0)
    segmented_person = np.copy(cv_image)
    segmented_person[mask] = resized_image[mask]

    mask_background = (cv_image == 255)
    background = np.copy(cv_image)
    background[mask_background] = resized_image[mask_background]
    background[np.where((background == [0,0,0]).all(axis = 2))] = [255,255,255]
    
    print(i)
    ROI = region_of_interest(rgb)
    foreground_blurscore, background_blurscore = blurriness(segmented_person,background)
    brightness_score = brightness(images[i])
    sigma_noise = noise(resized_image)
    colorfulness_score = colorfulness(resized_image)
    contrast_score = contrast(resized_image) 
    ROT_score , Area_ratio = ROT(cv_image)
    pot, land = potrait_or_landscape(cv_image)
    print('-----------------------------------------------------------')
    with open(filename,'a', newline = '') as f:
        fieldnames = ['ROI', 'Foreground Blur Score', 'Background blur score', 'Brightness Score', 'Noise score', 'Simplicity score', 'Contrast score', 'Area Ratio', 'Rule of third score','Potrait mode', 'Landscape Mode']
        writer = csv.DictWriter(f,fieldnames=fieldnames)
        if f.tell()==0:
            writer.writeheader()
        writer.writerow({'ROI':ROI, 'Foreground Blur Score':foreground_blurscore, 'Background blur score':background_blurscore, 'Brightness Score':brightness_score,'Noise score':sigma_noise, 'Simplicity score':colorfulness_score, 'Contrast score':contrast_score, 'Area Ratio':Area_ratio,'Rule of third score':ROT_score, 'Potrait mode':pot, 'Landscape Mode':land})
    





0
Region of interest is present i.e ROI = 1
Blur score of the foreground is  819.9538488375471
Blur score of the background is  527.4066371413116
Brightness score of the image is  93.59654405590234
the noise score of the image is  1.617109287446782
Colourfullness score of the image is  20.26175607142662
the contrast score of the image is  55.550555625158204
Ratio of area of foreground to the area of image is  0.543822388548951
The rule of third score of the image is  0.025098040202713834
the image is in landscape mode
-----------------------------------------------------------
1
Region of interest is present i.e ROI = 1
Blur score of the foreground is  599.0775045878013
Blur score of the background is  508.2658051723631
Brightness score of the image is  138.2084960991423
the noise score of the image is  0.6520321216473214
Colourfullness score of the image is  23.769462682628703
the contrast score of the image is  89.05078789470453
Ratio of area of foreground to the area of image is  0.

14
Region of interest is present i.e ROI = 1
Blur score of the foreground is  1572.8652478223667
Blur score of the background is  869.3810770362616
Brightness score of the image is  123.37819415452053
the noise score of the image is  6.439316781815712
Colourfullness score of the image is  38.398560878010464
the contrast score of the image is  59.017928961605065
Ratio of area of foreground to the area of image is  0.8089218139648438
The rule of third score of the image is  0.03559569393367285
the image is a square image
-----------------------------------------------------------
15
Region of interest is present i.e ROI = 1
Blur score of the foreground is  1543.4887182107195
Blur score of the background is  1624.543767302297
Brightness score of the image is  94.07598134642001
the noise score of the image is  1.2237795217147396
Colourfullness score of the image is  31.11220193368939
the contrast score of the image is  48.59373299094403
Ratio of area of foreground to the area of image is  

28
Region of interest is present i.e ROI = 1
Blur score of the foreground is  581.1029945255723
Blur score of the background is  648.0288988070097
Brightness score of the image is  94.5862562370805
the noise score of the image is  1.469594812839596
Colourfullness score of the image is  28.94554578926951
the contrast score of the image is  46.98829786066077
Ratio of area of foreground to the area of image is  0.7425384521484375
The rule of third score of the image is  0.0333312016901244
the image is a square image
-----------------------------------------------------------
29
Region of interest is present i.e ROI = 1
Blur score of the foreground is  970.3519704574344
Blur score of the background is  939.4667222017051
Brightness score of the image is  76.36152008673606
the noise score of the image is  3.0214927593737912
Colourfullness score of the image is  39.55715457960577
the contrast score of the image is  69.24732753958587
Ratio of area of foreground to the area of image is  0.68450

42
Region of interest is present i.e ROI = 1
Blur score of the foreground is  862.015808059834
Blur score of the background is  844.3623114523944
Brightness score of the image is  77.23375303638258
the noise score of the image is  1.1715548615919302
Colourfullness score of the image is  52.226977695083235
the contrast score of the image is  35.417199130817174
Ratio of area of foreground to the area of image is  0.6070480346679688
The rule of third score of the image is  0.027824733676017642
the image is a square image
-----------------------------------------------------------
43
Region of interest is present i.e ROI = 1
Blur score of the foreground is  1030.2018324829062
Blur score of the background is  1372.3191827709713
Brightness score of the image is  85.68426883290716
the noise score of the image is  3.5515104518309033
Colourfullness score of the image is  11.248165557224997
the contrast score of the image is  58.35795077868942
Ratio of area of foreground to the area of image is 

56
Region of interest is present i.e ROI = 1
Blur score of the foreground is  254.0649743380764
Blur score of the background is  663.8348460140079
Brightness score of the image is  80.61359646972213
the noise score of the image is  1.6823417540756966
Colourfullness score of the image is  63.93244579635255
the contrast score of the image is  56.94973782117585
Ratio of area of foreground to the area of image is  0.015619272360703812
The rule of third score of the image is  0.0006916496296285682
the image is in landscape mode
-----------------------------------------------------------
57
Region of interest is present i.e ROI = 1
Blur score of the foreground is  1387.4779119953148
Blur score of the background is  1257.9336446686577
Brightness score of the image is  71.77042821022114
the noise score of the image is  1.6434985010911904
Colourfullness score of the image is  32.071132403614776
the contrast score of the image is  35.50840102893914
Ratio of area of foreground to the area of imag

...

the contrast score of the image is  43.32731869867119
Ratio of area of foreground to the area of image is  0.37896611590038315
The rule of third score of the image is  0.017475043036327458
the image is in potrait mode
-----------------------------------------------------------
733
Region of interest is present i.e ROI = 1
Blur score of the foreground is  1555.1554870849607
Blur score of the background is  1549.9411730002396
Brightness score of the image is  25.13344032281883
the noise score of the image is  0.8473164955888661
Colourfullness score of the image is  48.509569202127416
the contrast score of the image is  18.027566836940615
Ratio of area of foreground to the area of image is  0.57623046875
The rule of third score of the image is  0.027001628817427594
the image is in landscape mode
-----------------------------------------------------------
734
Region of interest is present i.e ROI = 1
Blur score of the foreground is  667.5723350321072
Blur score of the background is  632.23

the contrast score of the image is  84.82513876307995
Ratio of area of foreground to the area of image is  0.6337253182114883
The rule of third score of the image is  0.02802529668135446
the image is in landscape mode
-----------------------------------------------------------
747
Region of interest is present i.e ROI = 1
Blur score of the foreground is  1354.368843860424
Blur score of the background is  1744.2772733624554
Brightness score of the image is  99.15278201549542
the noise score of the image is  2.825221826410279
Colourfullness score of the image is  92.12329367634031
the contrast score of the image is  93.49565475132266
Ratio of area of foreground to the area of image is  0.0001587345406360424
The rule of third score of the image is  6.074165372412924e-06
the image is in potrait mode
-----------------------------------------------------------
748
Region of interest is present i.e ROI = 1
Blur score of the foreground is  1631.9134927557882
Blur score of the background is  12

the contrast score of the image is  34.60469717620541
Ratio of area of foreground to the area of image is  0.6355856393129771
The rule of third score of the image is  0.029089841838248726
the image is in potrait mode
-----------------------------------------------------------
761
Region of interest is present i.e ROI = 1
Blur score of the foreground is  1366.7744288536232
Blur score of the background is  1129.9099808409965
Brightness score of the image is  45.582296571420926
the noise score of the image is  2.1339066404067606
Colourfullness score of the image is  11.0672030705474
the contrast score of the image is  36.487542475129786
Ratio of area of foreground to the area of image is  0.7352802805656934
The rule of third score of the image is  0.03236959265025848
the image is in potrait mode
-----------------------------------------------------------
762
Region of interest is present i.e ROI = 1
Blur score of the foreground is  2336.908333848701
Blur score of the background is  2314.9

775
Region of interest is present i.e ROI = 1
Blur score of the foreground is  458.8168759164223
Blur score of the background is  470.45171498545756
Brightness score of the image is  112.9569829268008
the noise score of the image is  0.7271626561680691
Colourfullness score of the image is  51.7943138697766
the contrast score of the image is  55.43065662748226
Ratio of area of foreground to the area of image is  0.11412894061583578
The rule of third score of the image is  0.005257772205707176
the image is in potrait mode
-----------------------------------------------------------
776
Region of interest is present i.e ROI = 1
Blur score of the foreground is  62.30884712837838
Blur score of the background is  690.5414183036631
Brightness score of the image is  173.64600840190195
the noise score of the image is  2.410346703158436
Colourfullness score of the image is  68.43254597503876
the contrast score of the image is  58.13993416133877
Ratio of area of foreground to the area of image is 

789
Region of interest is present i.e ROI = 1
Blur score of the foreground is  1130.454965535126
Blur score of the background is  531.894455792376
Brightness score of the image is  112.70800474770586
the noise score of the image is  4.861931690289246
Colourfullness score of the image is  36.15402695580261
the contrast score of the image is  54.905744750241226
Ratio of area of foreground to the area of image is  0.7169763901654411
The rule of third score of the image is  0.03167750818644267
the image is in potrait mode
-----------------------------------------------------------
790
Region of interest is present i.e ROI = 1
Blur score of the foreground is  620.5901694600541
Blur score of the background is  611.8410954794017
Brightness score of the image is  94.05152305102435
the noise score of the image is  0.4223366679995877
Colourfullness score of the image is  37.76633685302416
the contrast score of the image is  68.53625079891329
Ratio of area of foreground to the area of image is  0

In [6]:
# rgb = deeplab(images[0])

# cv_image = cv2.cvtColor(np.array(rgb), cv2.COLOR_RGB2BGR)
# h,w,n = cv_image.shape
# resized_image = cv2.resize(images[0],(w,h))

# mask = (cv_image == 0)
# segmented_person = np.copy(cv_image)
# segmented_person[mask] = resized_image[mask]

# mask_background = (cv_image == 255)
# background = np.copy(cv_image)
# background[mask_background] = resized_image[mask_background]
# background[np.where((background == [0,0,0]).all(axis = 2))] = [255,255,255]


# ROI = region_of_interest(rgb)
# foreground_blurscore, background_blurscore = blurriness(segmented_person,background)
# brightness_score = brightness(images[0])
# sigma_noise = noise(resized_image)
# colorfulness_score = colorfulness(resized_image)
# contrast_score = contrast(resized_image) 
# ROT_score , Area_ratio = ROT(cv_image)
# potrait,landscape = potrait_or_landscape(images[0])


In [51]:
import pandas as pd
url = "C:/Users/Akhil Jalla/Desktop/Final Project/Final_part1.csv"
names = ['ROI', 'Foreground Blur Score', 'Background blur score', 'Brightness Score', 'Noise score', 'Simplicity score', 'Contrast score','Area Ratio','Rule of third score','Potrait mode','Landscape Mode','Output']
features = pd.read_csv(url, skiprows=1, names=names)

In [52]:
labels = np.array(features['Output'])
features= features.drop('Output', axis = 1)
feature_list = list(features.columns)

from sklearn.model_selection import train_test_split

train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.20, random_state = 42)

from sklearn.ensemble import RandomForestRegressor

from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=10)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(train_features,train_labels)

y_pred=clf.predict(test_features)

print(clf.score(test_features,test_labels))

0.865625


In [53]:
# prediction = model.predict([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]])

In [54]:
test_images = []
high_quality_images = []
for f in glob.glob("C:/Users/Akhil Jalla/Desktop/Final Project/check/*.jpg"):
    image = cv2.imread(f)
    test_images.append(image)
print(len(test_images))

2


In [55]:
for i in range(len(test_images)):
    rgb = deeplab(test_images[i])
    cv_image = cv2.cvtColor(np.array(rgb), cv2.COLOR_RGB2BGR)
    h,w,n = cv_image.shape
    resized_image = cv2.resize(test_images[i],(w,h))
    
    mask = (cv_image == 0)
    segmented_person = np.copy(cv_image)
    segmented_person[mask] = resized_image[mask]

    mask_background = (cv_image == 255)
    background = np.copy(cv_image)
    background[mask_background] = resized_image[mask_background]
    background[np.where((background == [0,0,0]).all(axis = 2))] = [255,255,255]
    
    print(i)
    ROI = region_of_interest(rgb)
    foreground_blurscore, background_blurscore = blurriness(segmented_person,background)
    brightness_score = brightness(test_images[i])
    sigma_noise = noise(resized_image)
    colorfulness_score = colorfulness(resized_image)
    contrast_score = contrast(resized_image) 
    ROT_score , Area_ratio = ROT(cv_image)
    pot, land = potrait_or_landscape(cv_image)
    
    prediction = clf.predict([[ROI, foreground_blurscore, background_blurscore, brightness_score, sigma_noise, colorfulness_score, contrast_score, Area_ratio, ROT_score, pot, land]])
    print(prediction)
    if prediction == 1:
        high_quality_images.append(test_images[i])

0
Region of interest is present i.e ROI = 1
Blur score of the foreground is  1662.14373739195
Blur score of the background is  1599.1872774617661
Brightness score of the image is  23.878332052714484
the noise score of the image is  2.3055346185585797
Colourfullness score of the image is  16.341759759634886
the contrast score of the image is  21.171731919554915
Ratio of area of foreground to the area of image is  0.5550941623900293
The rule of third score of the image is  0.024613446088554883
the image is in landscape mode
[0]
1
Region of interest is present i.e ROI = 1
Blur score of the foreground is  780.7208747424427
Blur score of the background is  469.97337739153784
Brightness score of the image is  146.01911526833612
the noise score of the image is  1.2666393378609004
Colourfullness score of the image is  29.558419288675957
the contrast score of the image is  58.53559482699403
Ratio of area of foreground to the area of image is  0.3014896373056995
The rule of third score of the im

In [18]:
# cv2.imshow('image1',high_quality_images[0])
# cv2.waitKey(10000)
# cv2.destroyAllWindows()

In [45]:
# Referenced from https://www.learnopencv.com/image-quality-assessment-brisque/
# for i in range(len(high_quality_images)):
#     gray_image = cv2.cvtColor(high_quality_images[i], cv2.COLOR_BGR2GRAY) 
#     img_blur = cv2.GaussianBlur(gray_image, (7, 7), 1.166) 
#     blur_sq = img_blur**2
#     noise = cv2.GaussianBlur(gray_image * gray_image, (7, 7), 1.166) 
#     noise = (noise - blur_sq) ** 0.5
#     noise = noise + 1.0/255 
#     MCSN = (gray_image-img_blur)/noise 

#     pair_wise = [[0,1], [1,0], [1,1], [-1,1]]
#     for j in range(len(pair_wise)):
#         array = MCSN
#         required_shift = pair_wise[j]
#         new = np.float32([[1, 0, required_shift[1]], [0, 1, required_shift[0]]])
#         ShiftArr = cv2.warpAffine(array, new, (MCSN.shape[1], MCSN.shape[0]))
#     clf1 = svmutil.svm_load_model("allmodel")
#     x, idx = gen_svm_nodearray(x[1:], isKernel=(clf1.param.kernel_type == PRECOMPUTED))
#     qualityscore = svmutil.libsvm.svm_predict_probability(clf1, x, dec_values)
    
                                  
                  
    

NameError: name 'svmutil' is not defined